In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import IFrame, display
InteractiveShell.ast_node_interactivity = "all"

# Probabilities to cluster algorithm

A notebook to hash out this algorithm and check it works.

Will hopefully turn into a unit test too, hence CVSs into version control.

In [2]:
from src import locations as loc

import pandas as pd
import duckdb
from pathlib import Path

In [3]:
clus = pd.read_csv(Path(loc.PROJECT_DIR, "test", "clusters.csv"))
prob = pd.read_csv(Path(loc.PROJECT_DIR, "test", "probabilities.csv"))
val = pd.read_csv(Path(loc.PROJECT_DIR, "test", "validate.csv"))

In [85]:
duckdb.sql("""
    select
        *
    from
        prob;
""")

┌───────┬───────────┬─────────┬──────────────┬────────┬─────────────┐
│ uuid  │ link_type │ cluster │      id      │ source │ probability │
│ int64 │  varchar  │  int64  │   varchar    │ int64  │   double    │
├───────┼───────────┼─────────┼──────────────┼────────┼─────────────┤
│     1 │ link      │       0 │ will_inc_t1  │      1 │         1.0 │
│     2 │ link      │       1 │ will_inc_t2  │      2 │         0.9 │
│     3 │ link      │       2 │ will_inc_t2  │      2 │         0.7 │
│     4 │ link      │       3 │ will_inc_t2  │      2 │         0.4 │
│     5 │ link      │       4 │ will_inc_t2  │      2 │         0.2 │
│     6 │ link      │       1 │ will_inc_t3  │      3 │         0.8 │
│     7 │ link      │       2 │ will_inc_t3  │      3 │         0.7 │
│     8 │ link      │       3 │ will_inc_t3  │      3 │         0.1 │
│     9 │ link      │       4 │ will_inc_t3  │      3 │         0.3 │
│    10 │ link      │       1 │ will_inc_t4  │      4 │        0.75 │
│     · │  ·        

In [86]:
duckdb.sql("""
    select
        *
    from
        clus;
""")

┌───────┬─────────┬──────────────┬────────┬───────┬───────┐
│ uuid  │ cluster │      id      │ source │ n_seq │ n_par │
│ int64 │  int64  │   varchar    │ int64  │ int64 │ int64 │
├───────┼─────────┼──────────────┼────────┼───────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │     0 │
│     2 │       1 │ will_inc_t2  │      2 │     1 │     1 │
│     3 │       1 │ will_inc_t3  │      3 │     2 │     1 │
│     4 │       1 │ will_inc_t4  │      4 │     3 │     1 │
│     5 │       2 │ leo_inc_t1   │      1 │     0 │     0 │
│     6 │       2 │ leo_inc_t2   │      2 │     1 │     1 │
│     7 │       2 │ leo_inc_t4   │      4 │     3 │     1 │
│     8 │       3 │ pedro_inc_t1 │      1 │     0 │     0 │
│     9 │       3 │ pedro_inc_t2 │      2 │     1 │     1 │
│     9 │       3 │ pedro_inc_t4 │      4 │     3 │     1 │
│     9 │       4 │ sarah_inc_t1 │      1 │     0 │     0 │
│     9 │       4 │ sarah_inc_t2 │      2 │     1 │     1 │
│     9 │       4 │ sarah_inc_t3 │      

In [87]:
duckdb.sql("""
    select
        *
    from
        val;
""")

┌───────┬─────────┬──────────────┬────────┬─────────┬─────────┐
│ uuid  │ cluster │      id      │ source │  user   │  match  │
│ int64 │  int64  │   varchar    │ int64  │ varchar │ boolean │
├───────┼─────────┼──────────────┼────────┼─────────┼─────────┤
│     1 │       3 │ pedro_inc_t2 │      2 │ user01  │ true    │
│     2 │       4 │ sarah_inc_t3 │      3 │ user01  │ true    │
└───────┴─────────┴──────────────┴────────┴─────────┴─────────┘

## v1

Having done some fiddling below, here's the core SQL.

Step 1: instantiate clusters (or already have a cluster table)
Step 2: run this to add any new clusters the probabilities table now holds

Note this notebook DOESN'T handle adding unmatched dimensions to the clusters table as new clusters.

Params:

* n: your current stage in the 🐙blocktopus
* threshold: the point where we consider a probability a valid match. For parallel, note this means all tables will use the same value, which might not be the optimal value across all tables. In this instance, consider making it an additive table

In [ ]:
sql = """
    select
        nextval('uuid') as uuid, -- Create UUID in an appropriate way for Postgres
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_init on
            clus_init.id = prob.id
            and clus_init.source = prob.source
        where 
            probability > 0.7 -- Should be set by calling function
            and link_type = 'link'
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_init
"""

## v2

Woah. v1 is completely wrong. I thought making my IDs a bit like names would be helpful but it's just confused me -- the problem is the same ID being matched to loads of clusters and the interactions that emerge from that.

Let's recall why I thought this needed recursion:

* "will_inc" is 0.9 for cluster 1 and 0.8 for cluster 2
* "wedro_inc" is 0.8 for cluster 1 and 0.75 for cluster 2

In the current setup, "will_inc" goes to cluster 1 and cluster 2 is unmatched.

What we want is "will_inc" to go to cluster 1, and the second best option, "wedro_inc", to go to cluster 2.

There's some other stuff too.

* How do we handle validated clusters?
* Swear there was something else...

On validated, I say we add them first -- per round, not in a blob. Let's make a dummy validated table and add its conclusions to the clusters.

# Code

## v2

Now dealing with:

* Second best matches when the first best masked it
* Validated clusters

### v2.0

Without recursion or deletion.

In [5]:
clus_init = duckdb.sql("""
    drop sequence if exists uuid;
    drop sequence if exists cluster;
    create sequence uuid start 1;
    create sequence cluster start 1;
    select
        nextval('uuid') as uuid,
        nextval('cluster') as cluster,
        id,
        source,
        0 as n,
    from
        prob
    where
        cluster = 0
""")
clus_init

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [6]:
clus_val = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from
        val
    where 
        source in (
            select
                source
            from
                prob
        )
    union
    select
        *
    from
        clus_init
""")
clus_val

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│     5 │       3 │ pedro_inc_t2 │      2 │     1 │
│     6 │       4 │ sarah_inc_t3 │      3 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [111]:
duckdb.sql("""
    select
        distinct on (agg1.id, agg1.source)
        *
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_val cl on
            cl.id = prob.id
            and cl.source = prob.source
        anti join clus_val cl on
            cl.cluster = prob.cluster
            and cl.source = prob.source
        where 
            probability >= 0.7
        order by
            probability desc
    ) agg1
    order by
        agg1.probability desc
""")

┌───────┬───────────┬─────────┬──────────────┬────────┬─────────────┐
│ uuid  │ link_type │ cluster │      id      │ source │ probability │
│ int64 │  varchar  │  int64  │   varchar    │ int64  │   double    │
├───────┼───────────┼─────────┼──────────────┼────────┼─────────────┤
│     2 │ link      │       1 │ will_inc_t2  │      2 │         0.9 │
│    24 │ link      │       2 │ leo_inc_t4   │      4 │         0.9 │
│    38 │ link      │       3 │ pedro_inc_t4 │      4 │         0.9 │
│    16 │ link      │       2 │ leo_inc_t2   │      2 │         0.8 │
│     6 │ link      │       1 │ will_inc_t3  │      3 │         0.8 │
│    44 │ link      │       4 │ sarah_inc_t2 │      2 │         0.8 │
│    10 │ link      │       1 │ will_inc_t4  │      4 │        0.75 │
└───────┴───────────┴─────────┴──────────────┴────────┴─────────────┘

In [112]:
clus_1 = duckdb.sql("""
    select
        distinct on (agg.id, agg.source)
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_val cl on
            cl.id = prob.id
            and cl.source = prob.source
        anti join clus_val cl on
            cl.cluster = prob.cluster
            and cl.source = prob.source
        where 
            probability >= 0.7
        order by
            probability desc
    ) agg
    union
    select
        *
    from
        clus_val
""")
clus_1

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│    13 │       3 │ pedro_inc_t2 │      2 │     1 │
│    14 │       4 │ sarah_inc_t3 │      3 │     1 │
│    17 │       1 │ will_inc_t2  │      2 │     1 │
│    18 │       2 │ leo_inc_t4   │      4 │     1 │
│    19 │       3 │ pedro_inc_t4 │      4 │     1 │
│    20 │       2 │ leo_inc_t2   │      2 │     1 │
│    21 │       1 │ will_inc_t3  │      3 │     1 │
│    23 │       4 │ sarah_inc_t2 │      2 │     1 │
│    24 │       1 │ will_inc_t4  │      4 │     1 │
├───────┴─────────┴──────────────┴────────┴───────┤
│ 13 rows                               5 columns │
└───────────

In [113]:
duckdb.sql("""
    select
        distinct on (agg.id, agg.source)
        agg.*
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_1 cl on
            cl.id = prob.id
            and cl.source = prob.source
        anti join clus_1 cl on
            cl.cluster = prob.cluster
            and cl.source = prob.source
        where 
            probability >= 0.7
        order by
            probability desc
    ) agg
""")
duckdb.sql("""
    select
        count(distinct id),
        count(distinct concat(cluster, source)),
        count(*)
    from
        clus_1
""")
duckdb.sql("""
    select
        *
    from
        clus_1
    order by
        cluster
""")

┌───────┬───────────┬─────────┬──────────────┬────────┬─────────────┐
│ uuid  │ link_type │ cluster │      id      │ source │ probability │
│ int64 │  varchar  │  int64  │   varchar    │ int64  │   double    │
├───────┼───────────┼─────────┼──────────────┼────────┼─────────────┤
│    52 │ link      │       4 │ sarah_inc_t4 │      4 │         0.7 │
└───────┴───────────┴─────────┴──────────────┴────────┴─────────────┘

┌────────────────────┬───────────────────────────────────────────┬──────────────┐
│ count(DISTINCT id) │ count(DISTINCT concat("cluster", source)) │ count_star() │
│       int64        │                   int64                   │    int64     │
├────────────────────┼───────────────────────────────────────────┼──────────────┤
│                 13 │                                        13 │           13 │
└────────────────────┴───────────────────────────────────────────┴──────────────┘

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│    77 │       1 │ will_inc_t2  │      2 │     1 │
│    81 │       1 │ will_inc_t3  │      3 │     1 │
│    84 │       1 │ will_inc_t4  │      4 │     1 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│    78 │       2 │ leo_inc_t4   │      4 │     1 │
│    80 │       2 │ leo_inc_t2   │      2 │     1 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│    75 │       3 │ pedro_inc_t2 │      2 │     1 │
│    79 │       3 │ pedro_inc_t4 │      4 │     1 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│    76 │       4 │ sarah_inc_t3 │      3 │     1 │
│    83 │       4 │ sarah_inc_t2 │      2 │     1 │
├───────┴─────────┴──────────────┴────────┴───────┤
│ 13 rows                               5 columns │
└───────────

In [114]:
clus_2 = duckdb.sql("""
    select
        distinct on (agg.id, agg.source)
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_1 cl on
            cl.id = prob.id
            and cl.source = prob.source
        anti join clus_1 cl on
            cl.cluster = prob.cluster
            and cl.source = prob.source
        where 
            probability >= 0.7
        order by
            probability desc
    ) agg
    union
    select
        *
    from
        clus_1
""")
clus_2

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│   100 │       3 │ pedro_inc_t2 │      2 │     1 │
│   101 │       4 │ sarah_inc_t3 │      3 │     1 │
│   122 │       1 │ will_inc_t2  │      2 │     1 │
│   123 │       2 │ leo_inc_t4   │      4 │     1 │
│   124 │       3 │ pedro_inc_t4 │      4 │     1 │
│   125 │       2 │ leo_inc_t2   │      2 │     1 │
│   126 │       1 │ will_inc_t3  │      3 │     1 │
│   128 │       4 │ sarah_inc_t2 │      2 │     1 │
│   129 │       1 │ will_inc_t4  │      4 │     1 │
│   131 │       4 │ sarah_inc_t4 │      4 │     1 │
├───────┴─────────┴──────────────┴────────┴───────┤
│ 14 rows   

In [115]:
clus_3 = duckdb.sql("""
    select
        distinct on (agg.id, agg.source)
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_2 cl on
            cl.id = prob.id
            and cl.source = prob.source
        anti join clus_2 cl on
            cl.cluster = prob.cluster
            and cl.source = prob.source
        where 
            probability >= 0.7
        order by
            probability desc
    ) agg
    union
    select
        *
    from
        clus_2
""")
clus_3

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│   184 │       3 │ pedro_inc_t2 │      2 │     1 │
│   185 │       4 │ sarah_inc_t3 │      3 │     1 │
│   258 │       1 │ will_inc_t2  │      2 │     1 │
│   259 │       2 │ leo_inc_t4   │      4 │     1 │
│   260 │       3 │ pedro_inc_t4 │      4 │     1 │
│   261 │       2 │ leo_inc_t2   │      2 │     1 │
│   262 │       1 │ will_inc_t3  │      3 │     1 │
│   264 │       4 │ sarah_inc_t2 │      2 │     1 │
│   265 │       1 │ will_inc_t4  │      4 │     1 │
│   269 │       4 │ sarah_inc_t4 │      4 │     1 │
├───────┴─────────┴──────────────┴────────┴───────┤
│ 14 rows   

In [116]:
clus_check_l = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clus_3
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_r = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n_par::int as n
    from
        clus
    order by
        cluster,
        source,
        id,
        n_par
""")
clus_check_l.df().equals(clus_check_r.df())
clus_check_l.df().compare(clus_check_r.df())

True

Empty DataFrame
Columns: []
Index: []

### v2.1

Let's recurse 👹

In [11]:
duckdb.sql("""
    select
        uuid,
        cluster,
        id,
        source,
        n,
    from 
        clus_val
""")

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│    11 │       3 │ pedro_inc_t2 │      2 │     1 │
│    12 │       4 │ sarah_inc_t3 │      3 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [14]:
duckdb.sql("""
    with recursive clusters as (
        select
            uuid,
            cluster,
            id,
            source,
            n,
            1 as step
        from 
            clus_val
        union
        select distinct on (agg.id, agg.source)
            nextval('uuid') as uuid,
            agg.cluster,
            agg.id,
            agg.source,
            1 as n,
            step + 1
        from (
            select distinct on (p.cluster, p.source)
                p.*, (select max(step) from clusters) step
            from
                prob p
            where not exists (
                select  1             from
                    clusters cl
                where cl.id = p.id and cl.source = p.source and cl.step < 3
            )  and 
            not exists (
                select
                    1
                from
                    clusters cl
                where cl.cluster = p.cluster and cl.source = p.source and cl.step < 3
            )            
            and
                p.probability >= 0.7
            order by
                p.probability desc
        ) agg
    )
    select
        *
    from
        clusters   
""")

┌───────┬─────────┬──────────────┬────────┬───────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │ step  │
│ int64 │  int64  │   varchar    │ int64  │ int32 │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │     1 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │     1 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │     1 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │     1 │
│ 36712 │       3 │ pedro_inc_t2 │      2 │     1 │     1 │
│ 36713 │       4 │ sarah_inc_t3 │      3 │     1 │     1 │
│ 36714 │       1 │ will_inc_t2  │      2 │     1 │     2 │
│ 36715 │       2 │ leo_inc_t4   │      4 │     1 │     2 │
│ 36716 │       3 │ pedro_inc_t4 │      4 │     1 │     2 │
│ 36717 │       2 │ leo_inc_t2   │      2 │     1 │     2 │
│   ·   │       · │     ·        │      · │     · │     · │
│   ·   │       · │     ·        │      · │     · │     · │
│   ·   │       · │     ·        │      

### v2.2

Michał says that because the theoretical limit of the recursion is so high, we may end up with large blocking queries and hitting database settings limits. This is bad for performance and bugfixing. Suggests two tables and `delete returning`. I think this is a solid suggestion.

In [138]:
clus_init = duckdb.sql("""
    drop sequence if exists uuid;
    drop sequence if exists cluster;
    create sequence uuid start 1;
    create sequence cluster start 1;
    select
        nextval('uuid') as uuid,
        nextval('cluster') as cluster,
        id,
        source,
        0 as n,
    from
        prob
    where
        cluster = 0
""")
clus_init

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [139]:
clus_val = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from
        val
    where 
        source in (
            select
                source
            from
                prob
        )
    union
    select
        *
    from
        clus_init
""")
clus_val

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│     5 │       3 │ pedro_inc_t2 │      2 │     1 │
│     6 │       4 │ sarah_inc_t3 │      3 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [143]:
duckdb.sql("""
    drop sequence if exists uuid;
    drop sequence if exists cluster;
    create sequence uuid start 1;
    create sequence cluster start 1;
    select
        nextval('uuid') as uuid,
        nextval('cluster') as cluster,
        id,
        source,
        0 as n,
    from
        prob
    where
        cluster = 0
    union
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from
        val
    where 
        source in (
            select
                source
            from
                prob
        )
""")

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│     5 │       3 │ pedro_inc_t2 │      2 │     1 │
│     6 │       4 │ sarah_inc_t3 │      3 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

We want to:

1. Create a temporary `probability` table with every candidate value
2. Antijoin on `clusters` to get the bits we want to insert
3. If this contains values
    1. `delete returning` into `clusters`
    2. Go back to 2.
4. If the result is empty, done

In [140]:
duckdb.sql("""
    drop table if exists probabilities_temp;
    drop table if exists clusters_temp;
    
    create temp table probabilities_temp as
        select
            uuid,
            link_type,
            cluster,
            id,
            source,
            probability
        from
            prob prob
        where 
            prob.probability >= 0.7
            and cluster != 0
        order by
            probability desc;
    
    create temp table clusters_temp as
        select
            uuid,
            cluster,
            id,
            source,
            n,
        from
            clus_val;
""")

In [141]:
duckdb.sql("""
    select * from clusters_temp;
""")
duckdb.sql("""
    select * from probabilities_temp;
""")

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│     7 │       3 │ pedro_inc_t2 │      2 │     1 │
│     8 │       4 │ sarah_inc_t3 │      3 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

┌───────┬───────────┬─────────┬──────────────┬────────┬─────────────┐
│ uuid  │ link_type │ cluster │      id      │ source │ probability │
│ int64 │  varchar  │  int64  │   varchar    │ int64  │   double    │
├───────┼───────────┼─────────┼──────────────┼────────┼─────────────┤
│     2 │ link      │       1 │ will_inc_t2  │      2 │         0.9 │
│    24 │ link      │       2 │ leo_inc_t4   │      4 │         0.9 │
│    31 │ link      │       4 │ pedro_inc_t2 │      2 │         0.9 │
│    38 │ link      │       3 │ pedro_inc_t4 │      4 │         0.9 │
│    43 │ link      │       3 │ sarah_inc_t2 │      2 │         0.9 │
│    25 │ link      │       3 │ leo_inc_t4   │      4 │        0.85 │
│     6 │ link      │       1 │ will_inc_t3  │      3 │         0.8 │
│    16 │ link      │       2 │ leo_inc_t2   │      2 │         0.8 │
│    19 │ link      │       1 │ leo_inc_t3   │      3 │         0.8 │
│    26 │ link      │       4 │ leo_inc_t4   │      4 │         0.8 │
│    39 │ link      

In [142]:
clus_1 = duckdb.sql("""
    select
        distinct on (agg.id, agg.source)
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            probabilities_temp prob
        where 
            not exists (
                select
                    id,
                    source
                from
                    clusters_temp clus
                where
                    clus.id = prob.id
                    and clus.source = prob.source
            )
            or not exists (
                select
                    cluster,
                    source
                from
                    clusters_temp clus
                where
                    clus.cluster = prob.cluster
                    and clus.source = prob.source
            )
        order by
            probability desc
    ) agg;
""")
clus_1

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     9 │       1 │ will_inc_t2  │      2 │     1 │
│    10 │       2 │ leo_inc_t4   │      4 │     1 │
│    11 │       3 │ pedro_inc_t4 │      4 │     1 │
│    12 │       3 │ sarah_inc_t2 │      2 │     1 │
│    13 │       4 │ pedro_inc_t2 │      2 │     1 │
│    14 │       1 │ will_inc_t3  │      3 │     1 │
│    15 │       2 │ leo_inc_t2   │      2 │     1 │
│    17 │       1 │ will_inc_t4  │      4 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [145]:
len(clus_1.df().index)

8

In [135]:
duckdb.sql("""
    insert into clusters_temp 
    select
        uuid,
        cluster,
        id,
        source,
        n,
    from
        clus_1;

""")

In [114]:
duckdb.sql("""
    delete from probabilities_temp prob_temp
    where exists (
        select 
            cl.cluster,
            cl.id,
            cl.source
        from 
            clus_1 cl
        where
            (
                cl.id = prob_temp.id
                and cl.source = prob_temp.source
            ) or (
                cl.cluster = prob_temp.cluster
                and cl.source = prob_temp.source
            )
    );
""")

In [109]:
duckdb.sql("""
    delete from probabilities_temp prob_temp
    where exists (
        select 
            cl.cluster,
            cl.id,
            cl.source
        from 
            clus_1 cl
        where
            cl.id = prob_temp.id
            and cl.cluster = prob_temp.cluster
            and cl.source = prob_temp.source
    );
""")

In [136]:
duckdb.sql("""
    select * from clusters_temp;
""")
duckdb.sql("""
    select * from probabilities_temp;
""")

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ pedro_inc_t1 │      1 │     0 │
│     4 │       4 │ sarah_inc_t1 │      1 │     0 │
│    11 │       3 │ pedro_inc_t2 │      2 │     1 │
│    12 │       4 │ sarah_inc_t3 │      3 │     1 │
│    24 │       0 │ will_inc_t1  │      1 │     1 │
│    25 │       1 │ will_inc_t2  │      2 │     1 │
│    26 │       2 │ leo_inc_t4   │      4 │     1 │
│    27 │       3 │ pedro_inc_t4 │      4 │     1 │
│    28 │       3 │ sarah_inc_t2 │      2 │     1 │
│    29 │       4 │ pedro_inc_t2 │      2 │     1 │
│    30 │       1 │ will_inc_t3  │      3 │     1 │
│    31 │       2 │ leo_inc_t2   │      2 │     1 │
│    33 │       1 │ will_inc_t4  │      4 │     1 │
├───────┴───

┌───────┬───────────┬─────────┬──────────────┬────────┬─────────────┐
│ uuid  │ link_type │ cluster │      id      │ source │ probability │
│ int64 │  varchar  │  int64  │   varchar    │ int64  │   double    │
├───────┼───────────┼─────────┼──────────────┼────────┼─────────────┤
│     1 │ link      │       0 │ will_inc_t1  │      1 │         1.0 │
│    14 │ link      │       0 │ leo_inc_t1   │      1 │         1.0 │
│    27 │ link      │       0 │ pedro_inc_t1 │      1 │         1.0 │
│    40 │ link      │       0 │ sarah_inc_t1 │      1 │         1.0 │
│     2 │ link      │       1 │ will_inc_t2  │      2 │         0.9 │
│    24 │ link      │       2 │ leo_inc_t4   │      4 │         0.9 │
│    38 │ link      │       3 │ pedro_inc_t4 │      4 │         0.9 │
│    43 │ link      │       3 │ sarah_inc_t2 │      2 │         0.9 │
│    31 │ link      │       4 │ pedro_inc_t2 │      2 │         0.9 │
│    25 │ link      │       3 │ leo_inc_t4   │      4 │        0.85 │
│     · │  ·        

In [127]:
clus_check_l = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clusters_temp
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_r = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n_par::int as n
    from
        clus
    order by
        cluster,
        source,
        id,
        n_par
""")
clus_check_l.df().equals(clus_check_r.df())
clus_check_l.df().compare(clus_check_r.df())

False

cluster                  id               source          n      
      self other          self         other   self other self other
9      NaN   NaN  sarah_inc_t2  pedro_inc_t4    2.0   4.0  NaN   NaN
10     3.0   4.0  pedro_inc_t4  sarah_inc_t1    4.0   1.0  1.0   0.0
11     NaN   NaN  sarah_inc_t1  sarah_inc_t2    1.0   2.0  0.0   1.0
13     NaN   NaN  pedro_inc_t4  sarah_inc_t4    NaN   NaN  NaN   NaN

In [128]:
clus_check_l
clus_check_r

┌─────────┬──────────────┬────────┬───────┐
│ cluster │      id      │ source │   n   │
│  int64  │   varchar    │ int64  │ int32 │
├─────────┼──────────────┼────────┼───────┤
│       1 │ will_inc_t1  │      1 │     0 │
│       1 │ will_inc_t2  │      2 │     1 │
│       1 │ will_inc_t3  │      3 │     1 │
│       1 │ will_inc_t4  │      4 │     1 │
│       2 │ leo_inc_t1   │      1 │     0 │
│       2 │ leo_inc_t2   │      2 │     1 │
│       2 │ leo_inc_t4   │      4 │     1 │
│       3 │ pedro_inc_t1 │      1 │     0 │
│       3 │ pedro_inc_t2 │      2 │     1 │
│       3 │ sarah_inc_t2 │      2 │     1 │
│       3 │ pedro_inc_t4 │      4 │     1 │
│       4 │ sarah_inc_t1 │      1 │     0 │
│       4 │ sarah_inc_t3 │      3 │     1 │
│       4 │ pedro_inc_t4 │      4 │     1 │
├─────────┴──────────────┴────────┴───────┤
│ 14 rows                       4 columns │
└─────────────────────────────────────────┘

┌─────────┬──────────────┬────────┬───────┐
│ cluster │      id      │ source │   n   │
│  int64  │   varchar    │ int64  │ int32 │
├─────────┼──────────────┼────────┼───────┤
│       1 │ will_inc_t1  │      1 │     0 │
│       1 │ will_inc_t2  │      2 │     1 │
│       1 │ will_inc_t3  │      3 │     1 │
│       1 │ will_inc_t4  │      4 │     1 │
│       2 │ leo_inc_t1   │      1 │     0 │
│       2 │ leo_inc_t2   │      2 │     1 │
│       2 │ leo_inc_t4   │      4 │     1 │
│       3 │ pedro_inc_t1 │      1 │     0 │
│       3 │ pedro_inc_t2 │      2 │     1 │
│       3 │ pedro_inc_t4 │      4 │     1 │
│       4 │ sarah_inc_t1 │      1 │     0 │
│       4 │ sarah_inc_t2 │      2 │     1 │
│       4 │ sarah_inc_t3 │      3 │     1 │
│       4 │ sarah_inc_t4 │      4 │     1 │
├─────────┴──────────────┴────────┴───────┤
│ 14 rows                       4 columns │
└─────────────────────────────────────────┘

## v1

I've made the data more complicated so the below won't now evaluate to True.

### Parallel

Sometimes we might join several tables to `probabilities` at once, then add them to `clusters` together.

In [120]:
clus_init = duckdb.sql("""
    drop sequence if exists uuid;
    drop sequence if exists cluster;
    create sequence uuid start 1;
    create sequence cluster start 1;
    select
        nextval('uuid') as uuid,
        nextval('cluster') as cluster,
        id,
        source,
        0 as n,
    from
        prob
    where
        cluster = 0
""")
clus_init

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [121]:
clus_complete = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob
        anti join clus_init cl on
            cl.id = prob.id
            and cl.source = prob.source
        where 
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_init
""")

In [122]:
clus_check_l = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clus_complete
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_r = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n_par::int as n
    from
        clus
    order by
        cluster,
        source,
        id,
        n_par
""")
clus_check_l.df().equals(clus_check_r.df())
clus_check_l.df().compare(clus_check_r.df())

True

Empty DataFrame
Columns: []
Index: []

### Sequential

Sometimes we'll add one table to `probabilities`, then resolve to `clusters`, then do that over and over.

In [132]:
clus_init = duckdb.sql("""
    drop sequence if exists uuid;
    drop sequence if exists cluster;
    create sequence uuid start 1;
    create sequence cluster start 1;
    select
        nextval('uuid') as uuid,
        nextval('cluster') as cluster,
        id,
        source,
        0 as n,
    from
        prob
    where
        cluster = 0
""")
clus_init

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [133]:
prob_n1 = duckdb.sql("""
    select
        *
    from
        prob
    where
        source = 2
""")
prob_n2 = duckdb.sql("""
    select
        *
    from
        prob
    where
        source = 3
""")
prob_n3 = duckdb.sql("""
    select
        *
    from
        prob
    where
        source = 4
""")

In [134]:
clus_n1 = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        1 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob_n1 prob
        anti join clus_init cl on
            cl.id = prob.id
            and cl.source = prob.source
        where
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_init
""")
clus_n1

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
│     4 │       1 │ will_inc_t2  │      2 │     1 │
│     5 │       4 │ pedro_inc_t2 │      2 │     1 │
│     6 │       2 │ leo_inc_t2   │      2 │     1 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [135]:
clus_n2 = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        2 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob_n2 prob
        anti join clus_n1 cl on
            cl.id = prob.id
            and cl.source = prob.source
        where
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_n1
""")
clus_n2

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
│    10 │       1 │ will_inc_t2  │      2 │     1 │
│    11 │       4 │ pedro_inc_t2 │      2 │     1 │
│    12 │       2 │ leo_inc_t2   │      2 │     1 │
│    13 │       1 │ will_inc_t3  │      3 │     2 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [136]:
clus_n3 = duckdb.sql("""
    select
        nextval('uuid') as uuid,
        cluster,
        id,
        source,
        3 as n,
    from (
        select
            distinct on (prob.cluster, prob.source)
            prob.*
        from
            prob_n3 prob
        anti join clus_n2 on
            clus_n2.id = prob.id
            and clus_n2.source = prob.source
        where
            probability > 0.7
        order by
            probability desc,
            id desc
    )
    union
    select
        *
    from
        clus_n2
""")
clus_n3

┌───────┬─────────┬──────────────┬────────┬───────┐
│ uuid  │ cluster │      id      │ source │   n   │
│ int64 │  int64  │   varchar    │ int64  │ int32 │
├───────┼─────────┼──────────────┼────────┼───────┤
│     1 │       1 │ will_inc_t1  │      1 │     0 │
│     2 │       2 │ leo_inc_t1   │      1 │     0 │
│     3 │       3 │ sarah_inc_t1 │      1 │     0 │
│    14 │       1 │ will_inc_t2  │      2 │     1 │
│    15 │       4 │ pedro_inc_t2 │      2 │     1 │
│    16 │       2 │ leo_inc_t2   │      2 │     1 │
│    27 │       1 │ will_inc_t3  │      3 │     2 │
└───────┴─────────┴──────────────┴────────┴───────┘

In [137]:
clus_check_l1 = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clus_n2
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_l2 = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n::int as n
    from
        clus_n3
    order by
        cluster,
        source,
        id,
        n
""")
clus_check_r = duckdb.sql("""
    select
        cluster,
        id,
        source,
        n_seq::int as n
    from
        clus
    order by
        cluster,
        source,
        id,
        n_par
""")
clus_check_l1.df().equals(clus_check_r.df())
clus_check_l1.df().compare(clus_check_r.df())
clus_check_l2.df().equals(clus_check_r.df())
clus_check_l2.df().compare(clus_check_r.df())

True

Empty DataFrame
Columns: []
Index: []

True

Empty DataFrame
Columns: []
Index: []